In [ ]:
%matplotlib notebook
import numpy as np
from scipy import fftpack
import matplotlib.pyplot as plt
from skimage.feature import register_translation
import os
#os.environ['PYNX_PU'] = 'OpenCL.Iris'
from pynx.cdi.selection import *
from pynx.utils.pattern import siemens_star, fibonacci_urchin
import h5py as h5
h5.enable_ipython_completer()

In [ ]:
# 3D
if True:
    n = 64
    z, y, x = np.meshgrid(np.arange(n) / n,np.arange(n) / n, np.arange(n) / n, indexing='ij')

    d1 = fibonacci_urchin(dsize=n, r_max=n-10) #* np.exp(1j * (0.5 + 2 * x + 0.7 * y))
    d2 = fibonacci_urchin(dsize=n, r_max=n-11) * np.exp(1j * (-3 + 1 * x + 0.3 * y + 0.9 * z))
    d2 = np.flipud(d2)
else:
    h = h5.File('/Users/favre/data/201702-BraggCDI-Pt/alignment_S2280-2018-09-17T12-00-00_LLK000.366.cxi', 'r')
    d1 = h['entry_1/data_1/data'][()]
    h = h5.File('/Users/favre/data/201702-BraggCDI-Pt/alignment_S2280-2018-09-17T12-06-19_LLK000.368.cxi', 'r')
    d2 = h['entry_1/data_1/data'][()]
    # Crop to the same dimensions
    #d1 = d1[:40,:40,:40]
    #d2 = d2[:40,:40,:40]
    #print(d1.shape,d2.shape)

if True:
    d1g, d2g, r = match2(d1,d2,match_phase_ramp=True, verbose=2, use_gpu=True, match_orientation='center')
    print('\n\n')
    d1c, d2c, r = match2(d1,d2,match_phase_ramp=True, verbose=2, use_gpu=False, match_orientation='center')

In [ ]:
# 2D
n = 128

y, x = np.meshgrid(np.arange(n) / n, np.arange(n) / n, indexing='ij')

d1 = siemens_star(dsize=n, r_max=n-20, nb_rays=12, cheese_holes_nb=10) #* np.exp(1j * (0.5 + 2 * x + 0.7 * y))
d2 = siemens_star(dsize=n, r_max=n-20, nb_rays=12, cheese_holes_nb=10) * np.exp(1j * (-3 + 3 * x + 0.3 * y))
d2 = np.flipud(d2)

d1g, d2g, r = match2(d1,d2,match_phase_ramp=True, verbose=2, use_gpu=True, match_orientation='all')
print('\n\n')
d1c, d2c, r = match2(d1,d2,match_phase_ramp=True, verbose=2, use_gpu=False, match_orientation='all')

In [ ]:
# 2D - test manual correction from parameters
n = 128
dphi = 1.5
pixel_shift0 = 2, 5

y, x = np.meshgrid(np.arange(n) / n, np.arange(n) / n, indexing='ij')

d1 = siemens_star(dsize=n, r_max=n-20, nb_rays=12, cheese_holes_nb=10) #* np.exp(1j * (0.5 + 2 * x + 0.7 * y))
d2 = siemens_star(dsize=n, r_max=n-20, nb_rays=12, cheese_holes_nb=10) * np.exp(1j * (-3 + 1 * x + 0.5 * y))
print(register_translation(d1,d2))
d2 = ifftn(fourier_shift(fftn(d2), pixel_shift0)) * np.exp(1j * dphi)
print(register_translation(d1,d2))

d1g, d2g, r, flip, conj, pixel_shift, phase_shift = match2(d1,d2,match_phase_ramp=True, return_params=True)

print(flip, conj, pixel_shift, phase_shift)
print(register_translation(d1,d2))  # Original arrays are unmodified

# Manual correction
d2c = flipn(d2, flip)
d2c = ifftn(fourier_shift(fftn(d2), pixel_shift))
if conj:
    d2c = d2c.conj()
d2c = corr_phase(phase_shift, d2c)

d1g, d2g, r, flip, conj, pixel_shift, phase_shift = match2(d1,d2c,match_phase_ramp=True, return_params=True)
print(flip, conj, pixel_shift, phase_shift)  # Should all be zeros
